## Run pickprops module
Notebook to analyze 'locs_picked.hdf5' file in order to get 'locs_picked_props.hdf5' file containing kinetic parameters accroding to lbFCS and qPAINT analysis.



Define paths to data, i.e. '_locs_picked.hdf5' as obtained by picasso.render module. Multiple files can be processed by extending the list.

In [1]:
dir_names=[]
dir_names.extend([r'C:\Users\flori\Documents\data\sds_5nM'])

file_names=[]
file_names.extend([r'sds_5nM_locs_render_picked.hdf5']) 

Define input variables for lbFCS and qPAINT analysis and data saving. For evaluation of multiple files concentration list has to be extended.

In [2]:
conc=[5] # Imager concentration [nM]
ignore=1 # Ignore_dark value for qPAINT analysis
savename_ext='_props_ig%i'%(ignore) # File extension for processed file

#### Advanced 
omit_dist=True # If True all lists will be excluded for saving (recommended) 
kin_filter=True # If True automatic filtering will be applied (recommended)
NoPartitions=30 # Number of partitions for dask parallel computing

#### Dictonary content for .yaml file
props_info={'Generated by':'pickprops.get_props',
            'ignore':ignore,
            'omit_dist':omit_dist,
            'kin_filter': kin_filter}

Run analysis

In [6]:
#################################################### Load packages
import os #platform independent paths
import importlib
from IPython.display import clear_output

# Load user defined functions
import lbfcs.pickprops as props
import lbfcs.io as io
import lbfcs.pickprops_calls as props_call
# Reload modules
importlib.reload(props)
importlib.reload(props_call)

############################################################# Read locs, apply props & save locs
#### Create list of paths
path=[os.path.join(dir_names[i],file_names[i]) for i in range(0,len(file_names))]

#### Read-Apply-Save loop
for i in range(0,len(path)):
    #### File read in
    locs,locs_info=io.load_locs(path[i])
    
    #### Get number of frames
    NoFrames=locs_info[0]['Frames']
    
    #### Apply non-parallelized props
    locs_props=props.apply_props(locs,conc[i],NoFrames,ignore)
    
    #### Apply parallelized props
#    locs_props=props.apply_props_dask(locs,conc[i],NoFrames,ignore,NoPartitions)
    
    #### Drop objects for saving if omit=True
    if omit_dist:
        print('... removing distribution-lists from output')
        locs_props=locs_props.drop(['trace','tau','g','tau_b_dist','tau_d_dist'],axis=1)
    
    if kin_filter:
        print('... applying kinetic filter')
        locs_props=props._kin_filter(locs_props)
    
    #### Add nearest neigbour pick and distance
    print('... calculating nearest neighbour')
    locs_props=props_call.props_add_nn(locs_props)
    
    #### Save .hdf5 and .yaml of locs_props    
    io.save_locs(path[i].replace('.hdf5',savename_ext+'.hdf5'),
                    locs_props,
                    [locs_info,props_info],
                    mode='picasso_compatible')
clear_output()

C:\Users\flori\Anaconda3\envs\lbFCS\lib\site-packages\dask\context.py:23: UserWarning: The dask.set_options function has been deprecated. Please use dask.config.set instead
  warnings.warn("The dask.set_options function has been deprecated. "
C:/Users/flori/Documents/repos/lbFCS\lbfcs\pickprops.py:154: RuntimeWarning: invalid value encountered in log
  popt,pcov=scipy.optimize.curve_fit(ac_monoexp_lin,ac[1:10,0],-np.log(ac[1:10,1]-1),p0,bounds=(lowbounds,upbounds),method='trf')
C:/Users/flori/Documents/repos/lbFCS\lbfcs\pickprops.py:142: RuntimeWarning: divide by zero encountered in log
  g=t/tau-np.log(A)
C:/Users/flori/Documents/repos/lbFCS\lbfcs\pickprops.py:163: RuntimeWarning: invalid value encountered in log
  chisquare=np.sum(np.square(ac_monoexp_lin(ac[1:10,0],*popt)-np.log(ac[1:10,1]-1)))/(len(ac)-2)
C:\Users\flori\Anaconda3\envs\lbFCS\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Us

[                                        ] | 0% Completed | 15.2s


KeyboardInterrupt: 